In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql.types import TimestampType
import datetime
from pyspark.sql.functions import min, col, date_format
from pyspark.sql.types import StringType
from pyspark.sql.functions import sum
from pyspark.sql.functions import *
from pyspark.sql.functions import col
from pyspark.sql.functions import first
from pyspark.sql import functions as F
from pyspark.sql.functions import to_timestamp, desc
from pyspark.sql.functions import min, col, datediff, current_date
from pyspark.sql import Window

In [2]:
spark = SparkSession.builder.appName("Read CSV").getOrCreate()

item = spark.read.format("csv").option("header", "true").load("../3_STORAGE_LAYER/DATALAKEHOUSE_BRONZE_LAYER/items1.csv")
prod = spark.read.format("csv").option("header", "true").load("../3_STORAGE_LAYER/DATALAKEHOUSE_BRONZE_LAYER/production.csv")
warehouse = spark.read.format("csv").option("header", "true").load("../3_STORAGE_LAYER/DATALAKEHOUSE_BRONZE_LAYER/warehouse.csv")

23/01/15 20:59:39 WARN Utils: Your hostname, SameerAbdullas-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 192.168.29.57 instead (on interface en0)
23/01/15 20:59:39 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/01/15 20:59:40 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/01/15 20:59:41 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
23/01/15 20:59:41 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
23/01/15 20:59:41 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
23/01/15 20:59:41 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.


In [3]:
modwarehouse = warehouse

In [6]:
#string_to_datetime = udf(lambda x: datetime.datetime.strptime(x, "%d-%m-%Y"), TimestampType())
#modwarehouse = modwarehouse.withColumn("datetime", string_to_datetime("Registering Date"))

modwarehouse = modwarehouse.withColumn("datetime", to_timestamp("Registering Date", "dd-MM-yyyy"))
modwarehouse.columns
#modwarehouse.show()

['Entry No_',
 'Journal Batch Name',
 'Line No_',
 'Registering Date',
 'Location Code',
 'Zone Code',
 'Bin Code',
 'Description',
 'Item No_',
 'Quantity',
 'Qty_ (Base)',
 'Source Type',
 'Source Subtype',
 'Source No_',
 'Source Line No_',
 'Source Subline No_',
 'Source Document',
 'Source Code',
 'Reason Code',
 'No_ Series',
 'Bin Type Code',
 'Cubage',
 'Weight',
 'Journal Template Name',
 'Whse_ Document No_',
 'Whse_ Document Type',
 'Whse_ Document Line No_',
 'Entry Type',
 'Reference Document',
 'Reference No_',
 'User ID',
 'Variant Code',
 'Qty_ per Unit of Measure',
 'Unit of Measure Code',
 'Serial No_',
 'Lot No_',
 'Warranty Date',
 'Expiration Date',
 'Phys Invt Counting Period Code',
 'Phys Invt Counting Period Type',
 'Dedicated',
 'datetime']

In [7]:
datestep1 = modwarehouse.groupBy("Lot No_", "Bin Code", "Item No_").agg(min(col("datetime")).alias("Min_Registering_Date"))
datestep1 = datestep1.select("Lot No_", "Bin Code", "Item No_", date_format(col("Min_Registering_Date"), 'dd-MM-yyyy').alias("Min_Registering_Date"))
datestep1.show()
datestep1.printSchema()

+--------------+--------+---------------+--------------------+
|       Lot No_|Bin Code|       Item No_|Min_Registering_Date|
+--------------+--------+---------------+--------------------+
|IDS-21E2-00125|     TMG|       700536-4|          06-03-2021|
|IDS-21E2-00418|     IGS|        1000457|          10-02-2022|
|IDS-21E2-00372|     IGS|         117019|          11-04-2021|
|IDS-21E2-00564|     IGS|         112048|          20-03-2021|
|IDS-21E2-00625|     IGS|         102482|          20-01-2022|
|IDS-21E2-00560|     TMG|EII-CNS-OT-0025|          04-05-2021|
|IDS-21E2-00316|     IGS|         109456|          22-12-2022|
|IDS-21E2-00368|     IGS|         131536|          18-01-2022|
|IDS-21E2-00565|     TMG|         105581|          23-03-2021|
|IDS-21E2-00125|     IGS|         121876|          14-02-2022|
|IDS-21E2-00039|     IGS|         116912|          25-03-2021|
|IDS-21E2-00337|     TMG|         119852|          13-02-2021|
|IDS-21E2-00337|     TMG|         111742|          15-0

In [9]:
warehouse.createOrReplaceTempView("warehouse_table")
waretable = spark.table("warehouse_table")
print(type(waretable))
step1temp = waretable.groupBy("Lot No_", "Bin Code", "Item No_").agg(sum("Quantity").alias("Quantity"))
lis = step1temp.select("Quantity")
lis.show()
type(lis)
lis.count()

<class 'pyspark.sql.dataframe.DataFrame'>
+--------+
|Quantity|
+--------+
|  1900.0|
|     0.0|
|   375.0|
|   450.0|
|   250.0|
|  1925.0|
|   250.0|
|     0.0|
|     0.0|
|   200.0|
|   425.0|
|   200.0|
|   250.0|
|     0.0|
|   200.0|
|   250.0|
|   400.0|
|   250.0|
|     0.0|
|     0.0|
+--------+
only showing top 20 rows



4925

In [10]:
def func(lis):
    return sorted(lis.unique())[0]

udf_func = udf(func, StringType())
step1 = waretable.groupBy("Lot No_", "Bin Code", "Item No_").agg(min("Zone Code").alias("Zone Code"))
step1 = step1.join(step1temp, on=["Lot No_", "Bin Code", "Item No_"], how='inner')
step1 = step1.join(datestep1, on=["Lot No_", "Bin Code", "Item No_"], how='inner')
step1.show()

+--------------+--------+--------+---------+--------+--------------------+
|       Lot No_|Bin Code|Item No_|Zone Code|Quantity|Min_Registering_Date|
+--------------+--------+--------+---------+--------+--------------------+
|IDS-21E2-00028|     IGS| 1000457|      IGS|  1000.0|          19-05-2021|
|IDS-21E2-00028|     IGS|  102078|      IGS|   175.0|          08-08-2022|
|IDS-21E2-00028|     IGS|  102565|      IGS|   350.0|          28-01-2022|
|IDS-21E2-00028|     IGS|  102671|      IGS|   200.0|          13-12-2021|
|IDS-21E2-00028|     IGS|  104754|      IGS|     0.0|          24-02-2021|
|IDS-21E2-00028|     IGS|  105581|      IGS|  1175.0|          15-05-2021|
|IDS-21E2-00028|     IGS|  108259|      IGS|  1625.0|          15-03-2021|
|IDS-21E2-00028|     IGS|  111742|      IGS|  1200.0|          16-02-2021|
|IDS-21E2-00028|     IGS|  112048|      IGS|   825.0|          27-07-2021|
|IDS-21E2-00028|     IGS|  113014|      IGS|   250.0|          27-12-2021|
|IDS-21E2-00028|     IGS|

In [11]:
step1 = step1.withColumn("Min_Registering_Date", to_date(col("Min_Registering_Date"), 'dd-MM-yyyy'))
step1 = step1.withColumn("Days from Registration", datediff(current_date(),col("Min_Registering_Date")))

step1.show()


+--------------+--------+--------+---------+--------+--------------------+----------------------+
|       Lot No_|Bin Code|Item No_|Zone Code|Quantity|Min_Registering_Date|Days from Registration|
+--------------+--------+--------+---------+--------+--------------------+----------------------+
|IDS-21E2-00028|     IGS| 1000457|      IGS|  1000.0|          2021-05-19|                   606|
|IDS-21E2-00028|     IGS|  102078|      IGS|   175.0|          2022-08-08|                   160|
|IDS-21E2-00028|     IGS|  102565|      IGS|   350.0|          2022-01-28|                   352|
|IDS-21E2-00028|     IGS|  102671|      IGS|   200.0|          2021-12-13|                   398|
|IDS-21E2-00028|     IGS|  104754|      IGS|     0.0|          2021-02-24|                   690|
|IDS-21E2-00028|     IGS|  105581|      IGS|  1175.0|          2021-05-15|                   610|
|IDS-21E2-00028|     IGS|  108259|      IGS|  1625.0|          2021-03-15|                   671|
|IDS-21E2-00028|    

In [12]:
step1 = step1.filter(step1["Quantity"] > 0)
step1.count()
len(step1.columns)

7

In [13]:
step1.filter(col("Zone Code") == "IGS").filter(col("Item No_") == "102671").filter(col("Lot No_") == "IDS-21E2-00028").show()



+--------------+--------+--------+---------+--------+--------------------+----------------------+
|       Lot No_|Bin Code|Item No_|Zone Code|Quantity|Min_Registering_Date|Days from Registration|
+--------------+--------+--------+---------+--------+--------------------+----------------------+
|IDS-21E2-00028|     IGS|  102671|      IGS|   200.0|          2021-12-13|                   398|
+--------------+--------+--------+---------+--------+--------------------+----------------------+



In [28]:
step1.coalesce(1).write.format('com.databricks.spark.csv').options(header='true').save('../6_SERVING_LAYER/LAKEHOUSE_GOLD_LAYER/sc_step1.csv')


AnalysisException: path file:/Users/chemmi/Desktop/nest-data-hackathon/6_SERVING_LAYER/LAKEHOUSE_GOLD_LAYER/sc_step1.csv already exists.

In [15]:
print("Number of rows before removing NA values: ", item.count())
item = item.dropna(subset=['Production BOM No_'])
print("Number of rows after removing NA values: ", item.count())
item.show()
len(item.columns)

Number of rows before removing NA values:  12000
Number of rows after removing NA values:  8049
+--------+-----+--------------------+--------------------+-------------+--------------------+---------------------+----+-----------------------+---------+----------------+-------------------+----------------+----------------+----------+------------------------+--------+--------------+---------+-------------+-----------------+-----------------+---------------+--------------+------------------+
|     No_|No_ 2|         Description|  Search Description|Description 2|Base Unit of Measure|Price Unit Conversion|Type|Inventory Posting Group|Shelf No_|Item Disc_ Group|Allow Invoice Disc_|Statistics Group|Commission Group|Unit Price|Price_Profit Calculation|Profit _|Costing Method|Unit Cost|Standard Cost|Quoted Price(INR)|Quoted Price(FCY)|Quoted Currency|Standard Cost_|Production BOM No_|
+--------+-----+--------------------+--------------------+-------------+--------------------+-------------------

25

In [16]:
prod.show()
prod.count()
len(prod.columns)

+------------------+------------+--------+----+--------------------+--------+--------------------+--------+--------+----------+----------+----------------+-----------------+-------+------------+-------------+-----------+------+-----+------+-----+-------------------+------------+---------------+-------+--------+
|Production BOM No_|Version Code|Line No_|Type|         Description|     No_|Unit of Measure Code|Quantity|Position|Position 2|Position 3|Lead-Time Offset|Routing Link Code|Scrap _|Variant Code|Starting Date|Ending Date|Length|Width|Weight|Depth|Calculation Formula|Quantity per|Principal Input|Company|Division|
+------------------+------------+--------+----+--------------------+--------+--------------------+--------+--------+----------+----------+----------------+-----------------+-------+------------+-------------+-----------+------+-----+------+-----+-------------------+------------+---------------+-------+--------+
|       1000456-SMT|        null|11000000|   1|STATIC SHIELDE

26

In [17]:
step2 = item.alias("item").join(prod.alias("prod"), col("item.No_") == col("prod.No_"), "inner")
#step2.printSchema()
step2 = step2.select("item.No_").dropDuplicates()
step2 = step2.withColumnRenamed("No_", "Item No_")
step2.show()
step2.count()

+---------------+
|       Item No_|
+---------------+
|         149104|
|         118502|
|         124527|
|         700527|
|         123248|
|         106328|
|         129315|
|         111742|
|         148935|
|         111854|
|         116054|
|         145433|
|         131156|
|         104936|
|         112806|
|EII-CNS-CC-0002|
|         132646|
|         115870|
|         131319|
|         119027|
+---------------+
only showing top 20 rows



333

In [27]:
step2.coalesce(1).write.format('com.databricks.spark.csv').options(header='true').save('../6_SERVING_LAYER/LAKEHOUSE_GOLD_LAYER/sc_step2.csv')

In [18]:
# Perform left join on 'Item No_' column
step3 = step1.join(step2, on='Item No_', how='left')

# Drop columns that end with '_DROP'
drop_columns = [col for col in step3.columns if col.endswith('_DROP')]
step3 = step3.drop(*drop_columns)

step3.show()
# Print shape of step3
print((step3.count(), len(step3.columns)))


+--------+--------------+--------+---------+--------+--------------------+----------------------+
|Item No_|       Lot No_|Bin Code|Zone Code|Quantity|Min_Registering_Date|Days from Registration|
+--------+--------------+--------+---------+--------+--------------------+----------------------+
|  111742|IDS-21E2-00028|     IGS|      IGS|  1200.0|          2021-02-16|                   698|
|  131156|IDS-21E2-00028|     IGS|      IGS|   175.0|          2021-10-06|                   466|
|  105581|IDS-21E2-00028|     IGS|      IGS|  1175.0|          2021-05-15|                   610|
|  119192|IDS-21E2-00028|     IGS|      IGS|   200.0|          2022-09-17|                   120|
|  113014|IDS-21E2-00028|     IGS|      IGS|   250.0|          2021-12-27|                   384|
|  125168|IDS-21E2-00028|     IGS|      IGS|   200.0|          2021-12-24|                   387|
|  118270|IDS-21E2-00028|     IGS|      IGS|   425.0|          2021-10-17|                   455|
|  102671|IDS-21E2-0

In [26]:
step3.coalesce(1).write.format('com.databricks.spark.csv').options(header='true').save('../6_SERVING_LAYER/LAKEHOUSE_GOLD_LAYER/sc_step3.csv')


In [19]:
step4 = item.join(step3,item["No_"] == step3["Item No_"], "left_outer")



#step4 = item.where(item.No_.isin(step3.select("Item No_").collect()))
equals = step4.select(F.col("Item No_").eqNullSafe(F.col("No_"))).first()[0]
print(equals)
if(equals == True):
    step4 = step4.drop("No_")
print("Number of rows in step4: ", step4.count())

len(step4.columns)
print(step3.count())
item.count()
#item.show()

True
Number of rows in step4:  655789
3994


8049

In [25]:

step4.coalesce(1).write.format('com.databricks.spark.csv').options(header='true').save('../6_SERVING_LAYER/LAKEHOUSE_GOLD_LAYER/sc_step4.csv')

AnalysisException: path file:/Users/chemmi/Desktop/nest-data-hackathon/6_SERVING_LAYER/LAKEHOUSE_GOLD_LAYER/sc_step4.csv already exists.

In [25]:
step4.show()

+--------+--------------+--------+---------+--------+--------------------+----------------------+-----+--------------------+--------------------+-------------+--------------------+---------------------+----+-----------------------+---------+----------------+-------------------+----------------+----------------+----------+------------------------+--------+--------------+---------+-------------+-----------------+-----------------+---------------+--------------+------------------+
|Item No_|       Lot No_|Bin Code|Zone Code|Quantity|Min_Registering_Date|Days from Registration|No_ 2|         Description|  Search Description|Description 2|Base Unit of Measure|Price Unit Conversion|Type|Inventory Posting Group|Shelf No_|Item Disc_ Group|Allow Invoice Disc_|Statistics Group|Commission Group|Unit Price|Price_Profit Calculation|Profit _|Costing Method|Unit Cost|Standard Cost|Quoted Price(INR)|Quoted Price(FCY)|Quoted Currency|Standard Cost_|Production BOM No_|
+--------+--------------+--------+

In [24]:
step4.coalesce(1).write.format('com.databricks.spark.csv').options(header='true').save('../6_SERVING_LAYER/LAKEHOUSE_GOLD_LAYER/sc_step4.csv')

In [22]:
step4.createOrReplaceTempView("step4table")
step4table = spark.table("step4table")
sc_report1 = step4table.groupBy(["Item No_","Lot No_","Bin Code"]).agg(sum("Quantity").alias("Quantity")).distinct()
sc_report2 = step4table.groupBy(["Item No_","Lot No_","Bin Code"]).agg(sum("Quantity").alias("Quantity")).sort(desc("Quantity")).distinct().limit(10)
sc_report3 = step4.select("Item No_","Lot No_","Bin Code", "Days from Registration").sort(desc("Days from Registration")).distinct()
sc_report4 = step4table.groupBy("Bin Code").agg(sum("Quantity").alias("Quantity")).dropna().distinct()



sc_report1.show()
sc_report2.show()
sc_report3.show()
sc_report4.show()
print(sc_report1.count(),sc_report2.count(),sc_report3.count(),sc_report4.count())

+---------------+--------------+--------+--------+
|       Item No_|       Lot No_|Bin Code|Quantity|
+---------------+--------------+--------+--------+
|        1000457|IDS-21E2-00416|     IGS|136750.0|
|         108259|IDS-21E2-00325|     TMG|274050.0|
|         108259|IDS-21E2-00628|     IGS|430650.0|
|         110951|IDS-21E2-00368|     TMG|   400.0|
|         115132|IDS-21E2-00125|     TMG|  1400.0|
|         117232|IDS-21E2-00056|     TMG|  6475.0|
|         122106|IDS-21E2-00039|     IGS|  2975.0|
|         124257|IDS-21E2-00560|     IGS|  2275.0|
|         137932|IDS-21E2-00372|     TMG|  9100.0|
|         138531|IDS-21E2-00724|     TMG| 77750.0|
|         141388|IDS-21E2-00125|     IGS|  4500.0|
|         145340|IDS-21E2-00876|     TMG| 51000.0|
|       700536-2|IDS-21E2-00837|     IGS|  6825.0|
|EII-CNS-OT-0007|IDS-21E2-00125|     TMG|  1050.0|
|EII-CNS-OT-0030|IDS-21E2-00568|     IGS|  1600.0|
|        1000457|IDS-21E2-00912|     IGS|109400.0|
|        1000457|IDS-21E2-00328

In [23]:
sc_report1.coalesce(1).write.format('com.databricks.spark.csv').options(header='true').save('../6_SERVING_LAYER/LAKEHOUSE_GOLD_LAYER/sc_report1.csv')
sc_report2.coalesce(1).write.format('com.databricks.spark.csv').options(header='true').save('../6_SERVING_LAYER/LAKEHOUSE_GOLD_LAYER/sc_report2.csv')
sc_report3.coalesce(1).write.format('com.databricks.spark.csv').options(header='true').save('../6_SERVING_LAYER/LAKEHOUSE_GOLD_LAYER/sc_report3.csv')
sc_report4.coalesce(1).write.format('com.databricks.spark.csv').options(header='true').save('../6_SERVING_LAYER/LAKEHOUSE_GOLD_LAYER/sc_report4.csv')

